In [1]:
import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests


## Site Background

Sioux City, Iowa is located in the middle of the Missouri River basin in the central United States and borders both Nebraska and South Dakota. It features a humid continental climate with temperatures ranging from an average of 23 F in January to 74 F in July.  It exhibits grassland and riparian biomes and features a flat topography an elevation of roughly 1000 ft above sea level.  The city of Omaha features a populatoin of around 480,000 encompassing an area of around 132 sq miles.  The city is built around the Missouri River and thus features numerous levees and large concrete bridges close to the measuring site.

<img src="https://upload.wikimedia.org/wikipedia/commons/5/55/Sioux_City%2C_Iowa_%282022%29.jpg" width=700>

> Image Source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Sioux_City,_Iowa_(2022).jpg/)

# SITE MAP

In [3]:
#Initialize lat and long inputs
sg_lat = 42.48619444			
sg_lon = -96.4130833	

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # initial zoom level
    zoom_start= 15,
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], 
              popup="Missouri River South Bank at Nebraska / Iowa Border under Southland Verterans Memorial Bridge",
               icon=folium.Icon(color="green"),
               ).add_to(m)

# Display the map
m

## Data description
The following data represents the stream gauge hydrographs taken at a gauge on the southern bank of the Missouri River on the border of Nebraska and Iowa in Sioux City. It reprseents the daily maximum measurements during the water years from 2008 - 2022 and ranges from a low of around 10,000 cubic feet / sec (cfs) to a maximum of 192,000 cfs during a flood event in spring of 2011. Data was retrieved from the USGS National Water Dashboard. 

Citation: “Missouri River at Sioux City, IA.” USGS Water Data for the Nation, United States Geological Survery, 18 Sept. 2023, https://waterdata.usgs.gov/monitoring-location/06486000/#parameterCode=00065&amp;period=P7D&amp;showMedian=true. Accessed 18, Sept. 2023.

## Hydrographs
### Steady Flow regime from 2000 interrupted by 2 larger events during 2010's
The data shows an increasing in maximum discharge during the 1990's that settles into a fairly clam regime starting around water year 2000.  The flooding events from 2011 and 2019 can be seen clearly on this graph and in both years the water level stays elevated for several months before falling during the colder months.

In [4]:
# Store url for dataset in var
SiouxCity_stream_url = ("https://nwis.waterdata.usgs.gov/usa/nwis/uv/?cb_00060=on"
                "&format=rdb&site_no=06486000&legacy=1&period="
                "&begin_date=1991-10-01&end_date=2022-09-30")

# Retrive dataset and store in Var
SiouxCity_response = requests.get(SiouxCity_stream_url)



In [38]:
# Create initial dataframe
SiouxCity_stream_DF= pd.read_csv(
    BytesIO(SiouxCity_response.content),
    comment='#',
    delimiter='\t',
    skiprows=[29, 30],
    names=['Agency', 'Site No.', 'Date', 'Timezone', 'Discharge (cfs)', 'Approval Status'],
    index_col=['Date'],
    parse_dates=True,
)



# Resample for daily maximum and drop NaN values
SiouxCity_stream_dailymax_df = SiouxCity_stream_DF[['Discharge (cfs)']].resample('D').max()
SiouxCity_stream_dailymax_df.dropna()

SiouxCity_stream_dailymax_df.hvplot(x='Date', 
                        y='Discharge (cfs)', 
                        title='Missouri River Daily Max Discharge from 1991 - 2022', 
                        ylabel='Discharge (cfs)', 
                        xlabel='Date', 
                        width=800, 
                        height=400, 
                        fontsize=20, 
                        color='teal',
                        use_index=True)

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [Date]   (Discharge (cfs))

### Flooding Event in Spring 2019 results in elevated hydrograph until winter
In Spring of 2019, the river systems in the Central United States experienced substantial levels of flooding due to a snowmelt that occured earlier than usual that was unable to permeate into saturated soil.  This led to highly increased water levels that in some plades were almost orders larger than what was typical. In the below graph we see a sharp increase in the middle of March which drops just as quickly, but levels off at value around four times what it was during the winter months.  This elevated profile is marked by two other events in May and Septemeber, which seem to coincide with seasonal precipitation runoff. The hydrograph profile drops off to the steady value seen in the previous graph around December 2019.

In [37]:
SiouxCity_stream_dailymax_df['2018-10-01':'2020-10-01'].hvplot(x='Date', 
                        y='Discharge (cfs)', 
                        title='Missouri River Daily Max Discharge from 2018 - 2020', 
                        ylabel='Discharge (cfs)', 
                        xlabel='Date', 
                        width=800, 
                        height=400, 
                        fontsize=20, 
                        color='teal',
                        use_index=True)

/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/opt/conda/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [Date]   (Discharge (cfs))

In [ ]:
%%capture
%%bash
jupyter nbconvert <your-notebook-name>.ipynb --to html --no-input